In [1]:
import numpy as np
import pandas as pd
import scipy
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 7 environment'ı ortamı kuruyoruz
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Colab Notebooks/input"

In [5]:
!kaggle datasets download -d busranurok/flo-data-20k

  0% 0.00/656k [00:00<?, ?B/s]
100% 656k/656k [00:00<00:00, 149MB/s]


In [6]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  flo-data-20k.zip
  inflating: flo_data_20k.csv        


In [84]:
df_segmentation = pd.read_csv("flo_data_20k.csv",delimiter=',',index_col = 0)
df_segmentation.head()

,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
master_id,,,,,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [65]:
# prompt: sutun sil

df_segmentation.drop('first_order_date', axis=1, inplace=True)


In [67]:
df_segmentation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19945 entries, cc294636-19f0-11eb-8d74-000d3a38a36f to 740998d2-b1f7-11e9-89fa-000d3a38a36f
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_channel                      19945 non-null  object 
 1   last_order_channel                 19945 non-null  object 
 2   first_order_date                   19945 non-null  object 
 3   last_order_date                    19945 non-null  object 
 4   last_order_date_online             19945 non-null  object 
 5   last_order_date_offline            19945 non-null  object 
 6   order_num_total_ever_online        19945 non-null  float64
 7   order_num_total_ever_offline       19945 non-null  float64
 8   customer_value_total_ever_offline  19945 non-null  float64
 9   customer_value_total_ever_online   19945 non-null  float64
 10  interested_in_categories_12        19945 non-null  object

In [53]:
df_segmentation.isnull().sum()

order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [85]:
df_segmentation.columns

Index(['order_channel', 'last_order_channel', 'first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline', 'order_num_total_ever_online', 'order_num_total_ever_offline', 'customer_value_total_ever_offline', 'customer_value_total_ever_online', 'interested_in_categories_12'], dtype='object')

In [86]:
# prompt: veriy tara ve object veri tipinde bul ve oları  categori veri    veri tipine çevir

object_cols = df_segmentation.select_dtypes(include='object').columns.tolist()

for col in object_cols:
    df_segmentation[col] = df_segmentation[col].astype('category')

df_segmentation.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19945 entries, cc294636-19f0-11eb-8d74-000d3a38a36f to 740998d2-b1f7-11e9-89fa-000d3a38a36f
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   order_channel                      19945 non-null  category
 1   last_order_channel                 19945 non-null  category
 2   first_order_date                   19945 non-null  category
 3   last_order_date                    19945 non-null  category
 4   last_order_date_online             19945 non-null  category
 5   last_order_date_offline            19945 non-null  category
 6   order_num_total_ever_online        19945 non-null  float64 
 7   order_num_total_ever_offline       19945 non-null  float64 
 8   customer_value_total_ever_offline  19945 non-null  float64 
 9   customer_value_total_ever_online   19945 non-null  float64 
 10  interested_in_categories_12        19945 non-

In [56]:
#categorical_features, veri tipi ‘object’ olan sütunları seçer.

categorical_features = df_segmentation.select_dtypes(include=['object']).columns
#numerical_features ise veri tipi ‘float’ veya ‘int’ olan sütunları seçer.

numerical_features = df_segmentation.select_dtypes(include=['float', 'int']).columns

In [57]:
#numerical_features ise veri tipi ‘float’ veya ‘int’ olan sütunları seçer.
numerical_transformer = Pipeline(steps=[
    ('scalar', StandardScaler())
])
#categorical_features, veri tipi ‘object’ olan sütunları seçer.
categorical_transformer = Pipeline(steps=[
    ('scalar', OneHotEncoder())
])


# Bu nesne, sayısal ve kategorik özelliklere farklı dönüştürücüler uygulayarak, veri çerçevesini tek bir özellik matrisine dönüştürür.
# transformers parametresi, hangi dönüştürücünün hangi özelliklere uygulanacağını belirtir.
# Örneğin, (‘num’, numerical_transformer, numerical_features) demek, numerical_transformer’ın numerical_features’e uygulanacağı anlamına gelir.
# Aynı şekilde, (‘cat’, categorical_transformer, categorical_features) demek, categorical_transformer’ın categorical_features’e uygulanacağı anlamına gelir.


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

In [87]:
df_segmentation.head()

,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
master_id,,,,,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [88]:
# prompt: verileri tara ve onları label encoding yap

import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column in df_segmentation:
    if df_segmentation[column].dtype == np.number:
        continue
    df_segmentation[column] = le.fit_transform(df_segmentation[column])
df_segmentation.head()


<ipython-input-88-59876893a9d6>:7: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df_segmentation[column].dtype == np.number:


,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
master_id,,,,,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,0,4,2274,272,1644,672,4.0,1.0,139.99,799.38,30
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,0,3,947,262,1639,338,19.0,2.0,159.97,1853.58,23
69b69676-1a40-11ea-941b-000d3a38a36f,0,0,1936,181,1558,298,3.0,2.0,189.97,395.35,27
1854e56c-491f-11eb-806e-000d3a38a36f,0,0,2342,232,1609,635,1.0,1.0,39.99,81.98,4
d6ea1074-f1f5-11e9-9346-000d3a38a36f,1,1,1820,281,1658,178,1.0,1.0,49.99,159.99,16


In [89]:
df_segmentation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19945 entries, cc294636-19f0-11eb-8d74-000d3a38a36f to 740998d2-b1f7-11e9-89fa-000d3a38a36f
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_channel                      19945 non-null  int64  
 1   last_order_channel                 19945 non-null  int64  
 2   first_order_date                   19945 non-null  int64  
 3   last_order_date                    19945 non-null  int64  
 4   last_order_date_online             19945 non-null  int64  
 5   last_order_date_offline            19945 non-null  int64  
 6   order_num_total_ever_online        19945 non-null  float64
 7   order_num_total_ever_offline       19945 non-null  float64
 8   customer_value_total_ever_offline  19945 non-null  float64
 9   customer_value_total_ever_online   19945 non-null  float64
 10  interested_in_categories_12        19945 non-null  int64 

In [90]:
scaler = StandardScaler()
segmentation_std = scaler.fit_transform(df_segmentation)

# K-means Clustering

In [91]:

import random
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [93]:
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(segmentation_std)
    wcss.append(kmeans.inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
kmeans = KMeans()
ssd = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k).fit(df_segmentation)
    ssd.append(kmeans.inertia_)

plt.plot(K, ssd, "bx-")
plt.xlabel("Farklı K Değerlerine Karşılık SSE/SSR/SSD")
plt.title("Optimum Küme sayısı için Elbow Yöntemi")
plt.show()

kmeans = KMeans()
elbow = KElbowVisualizer(kmeans, k=(2, 10))
elbow.fit(df_segmentation)
elbow.elbow_value_
df_segmentation.info()

In [94]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)

In [95]:
kmeans.fit(segmentation_std)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4, random_state=42)

In [96]:
df_segm_kmeans = df_segmentation.copy()
df_segm_kmeans['Segment K-means'] = kmeans.labels_

In [97]:
df_segm_analysis = df_segm_kmeans.groupby(['Segment K-means']).mean()
df_segm_analysis

,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
Segment K-means,,,,,,,,,,,
0,1.912221,3.279788,1891.530200,234.384254,1515.859342,498.745123,2.070153,1.967215,274.068637,343.219317,21.800470
1,1.590038,3.966369,1004.142188,181.293742,728.598553,590.185185,2.283099,2.525756,345.335986,266.839476,22.719455
2,0.910150,1.251248,1049.394343,268.472546,1628.686356,383.937604,14.499168,2.125624,268.386148,2360.485549,17.342762
3,0.246702,0.290969,1805.767377,240.343734,1617.343734,320.301497,2.744546,1.641806,202.735632,448.271837,21.501395


In [98]:
df_segm_analysis['N Obs'] = df_segm_kmeans[['Segment K-means','order_channel']].groupby(['Segment K-means']).count()
df_segm_analysis['Prop Obs'] = df_segm_analysis['N Obs'] / df_segm_analysis['N Obs'].sum()

In [99]:
df_segm_analysis

,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,N Obs,Prop Obs
Segment K-means,,,,,,,,,,,,,
0,1.912221,3.279788,1891.530200,234.384254,1515.859342,498.745123,2.070153,1.967215,274.068637,343.219317,21.800470,8510,0.426673
1,1.590038,3.966369,1004.142188,181.293742,728.598553,590.185185,2.283099,2.525756,345.335986,266.839476,22.719455,2349,0.117774
2,0.910150,1.251248,1049.394343,268.472546,1628.686356,383.937604,14.499168,2.125624,268.386148,2360.485549,17.342762,1202,0.060266
3,0.246702,0.290969,1805.767377,240.343734,1617.343734,320.301497,2.744546,1.641806,202.735632,448.271837,21.501395,7884,0.395287
